In [16]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Namish\OneDrive\Documents\projects\ACM day6\flights.csv")
print("loaded file ok,hope its the right one")
df.head()


loaded file ok,hope its the right one


,flight_id,airline,origin,destination,dep_time,arr_time,delay_min,distance_km,seat_load_factor,cancelled
0,F0001,delta,atl,jfk,2024-08-01 06:35,2024-08-01 08:52,12.0,1220,84.3,False
1,F0002,United,LAX,sfo,2024-08-01 07:10,2024-08-01 08:28,-3.0,543,91.1,False
2,F0003,american,ord,dfw,2024-08-02 09:22,2024-08-02 11:58,25.0,1292,77.5,False
3,F0004,Southwest,dal,hOU,2024-08-02 10:05,2024-08-02 11:15,NaN,385,68.2,False
4,F0005,alaska,SEA,LAX,2024-08-02 13:40,2024-08-02 15:55,7.0,1545,88.9,False


In [17]:
df["dep_time"]=pd.to_datetime(df["dep_time"],errors="coerce")
df["arr_time"]=pd.to_datetime(df["arr_time"],errors="coerce")
df["dep_hour"]=df["dep_time"].dt.hour
df["arr_hour"]=df["arr_time"].dt.hour
for c in ["airline","origin","destination"]:
    df[c]=df[c].astype(str).str.strip().str.lower()
df["cancelled"]=df["cancelled"].astype(str).str.strip().str.lower().map({"true":True,"false":False})
print("types fixed a bit,strings cleaned")


types fixed a bit,strings cleaned


In [18]:
for col in ["delay_min","distance_km","seat_load_factor","dep_hour","arr_hour"]:
    df[col]=pd.to_numeric(df[col],errors="coerce")
    df[col]=df[col].fillna(df[col].median())
for col in ["airline","origin","destination"]:
    df[col]=df[col].fillna(df[col].mode().iloc[0])
df["cancelled"]=df["cancelled"].fillna(False)
print("missing stuff got filled")


missing stuff got filled


In [19]:
def iqr_clip(s,k=1.5):
    q1=s.quantile(0.25)
    q3=s.quantile(0.75)
    iqr=q3-q1
    low=q1-k*iqr
    high=q3+k*iqr
    return s.clip(lower=low,upper=high)
for c in ["delay_min","distance_km","seat_load_factor"]:
    df[c]=iqr_clip(df[c])
print("outliers got clipped a bit")


outliers got clipped a bit


In [20]:
df["cancelled"]=df["cancelled"].astype(int)
encoded=pd.get_dummies(df[["airline","origin","destination"]],drop_first=True)
processed=pd.concat([df[["flight_id","delay_min","distance_km","seat_load_factor","dep_hour","arr_hour","cancelled"]],encoded],axis=1)
print("encoding done")
processed.head()


encoding done


,flight_id,delay_min,distance_km,seat_load_factor,dep_hour,arr_hour,cancelled,airline_american,airline_delta,airline_frontier,...,destination_lax,destination_lga,destination_mco,destination_msy,destination_ord,destination_phx,destination_rsw,destination_san,destination_sea,destination_sfo
0,F0001,12.0,1220.0,84.3,6,8,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,F0002,-3.0,543.0,91.1,7,8,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,F0003,25.0,1292.0,77.5,9,11,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,F0004,14.5,385.0,68.2,10,11,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,F0005,7.0,1545.0,88.9,13,15,0,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [22]:
from sklearn.preprocessing import StandardScaler
scale_cols=["delay_min","distance_km","seat_load_factor","dep_hour","arr_hour"]
scaler=StandardScaler()
processed[scale_cols]=scaler.fit_transform(processed[scale_cols])
print("scaling finished,")
processed.head()


scaling finished,


,flight_id,delay_min,distance_km,seat_load_factor,dep_hour,arr_hour,cancelled,airline_american,airline_delta,airline_frontier,...,destination_lax,destination_lga,destination_mco,destination_msy,destination_ord,destination_phx,destination_rsw,destination_san,destination_sea,destination_sfo
0,F0001,-0.357142,0.106061,0.942477,-1.248214,-1.200138,0,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,F0002,-1.433953,-0.986085,1.585162,-0.985984,-1.200138,0,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,F0003,0.576094,0.222212,0.299793,-0.461524,-0.414021,0,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,F0004,-0.177674,-1.240973,-0.579172,-0.199295,-0.414021,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,F0005,-0.716079,0.630356,1.377234,0.587395,0.634134,0,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [24]:
processed.to_csv("flights_processed.csv",index=False)
print("saved flights_processed.csv,pls check it there,ok thanks")


saved flights_processed.cs
